In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as st
import statsmodels.api as sm
import numpy as np

from VaR_func import *
from tests import *

In [5]:
data = pd.read_csv(".\\data\\ropa.csv", index_col='Date', header=0)
data['returns'] = data.interpolate(type='linear').pct_change()
data.index = pd.to_datetime(data.index)
r = list(data['returns'])
r = r[1:]
r_test = r[-298:]
fitted_dist = pd.read_csv('fitted_distributions_best.csv', index_col=0)

In [3]:
dist_type = "t"
filename = './data_distfit/fitted_{}.csv'.format(dist_type)
df1 = pd.read_csv(filename, index_col=0)
#df1

,t_min,t_max,params,SSE,ecdf_quantiles,VaR_95,VaR_99,EVaR_95,EVaR_99
-1,0,798,"(6.080803251126445, 0.0015888318874874476, 0.0...",0.131604,[-1.30628881e-01 -1.21261119e-01 -8.23572838e-...,0.040322,0.064102,[0.02960006],[0.04773091]
0,0,499,"(5.360464598578685, 0.002028062353721335, 0.02...",0.098780,"[-0.1306288805140844, -0.12126111859385258, -0...",0.044278,0.071661,[0.03286755],[0.05364779]
1,1,500,"(5.321771569906347, 0.0021115908063192882, 0.0...",0.105857,"[-0.1306288805140844, -0.12126111859385258, -0...",0.044344,0.071808,[0.03295731],[0.05379429]
2,2,501,"(5.288460197098301, 0.002026474527667888, 0.02...",0.101235,"[-0.1306288805140844, -0.12126111859385258, -0...",0.044216,0.071732,[0.03285743],[0.05373046]
3,3,502,"(5.241928798602011, 0.002006924283874679, 0.02...",0.107570,"[-0.1306288805140844, -0.12126111859385258, -0...",0.044091,0.071651,[0.03278401],[0.0536861]
...,...,...,...,...,...,...,...,...,...
293,293,792,"(18.46934189425143, 0.00019678092206142114, 0....",0.141359,"[-0.08235728377794271, -0.07925833692240081, -...",0.037519,0.055073,[0.02631892],[0.04051626]
294,294,793,"(17.770366010399627, 0.00012607907540755143, 0...",0.139425,"[-0.08235728377794271, -0.07925833692240081, -...",0.037363,0.054965,[0.0262004],[0.04041402]
295,295,794,"(17.73884167806812, 0.0001625987601562807, 0.0...",0.147290,"[-0.08235728377794271, -0.07925833692240081, -...",0.037403,0.055011,[0.02623968],[0.0404568]
296,296,795,"(17.731184434662175, 0.00015986835169561135, 0...",0.145779,"[-0.08235728377794271, -0.07925833692240081, -...",0.037399,0.055008,[0.0262365],[0.04045385]


In [82]:
def summary_backtesting(df, pr=True):
    v95 = df['VaR_95'][1:]
    v99 = df['VaR_99'][1:]
    ev95 = df['EVaR_95'][1:]
    ev99 = df['EVaR_99'][1:]
    I_95 = []
    I_99 = []
    for i in range(len(r_test)):
        I_95.append(v95[i] < r_test[i])
        I_99.append(v99[i] < r_test[i])
    t = '       '
    res = {}
    res['Ku95'] = kupiec(np.array(I_95), .95)
    res['Ku99'] = kupiec(np.array(I_99), .99)
    if sum(I_95) != 0:
        res['Ch95'] = christoffersen(np.array(I_95))
    else: res['Ch95']= None
    if sum(I_99) != 0:
        res['Ch99'] = christoffersen(np.array(I_99))
    else: res['Ch99']= None
    res['Sc95'] = np.mean(scoring_quantile(r_test, v95, .95))
    res['Sc99'] = np.mean(scoring_quantile(r_test, v99, .99))
    res['ESc95'] = np.mean(scoring_expectile(r_test, ev95, .95))
    res['ESc99'] = np.mean(scoring_expectile(r_test, ev99, .99))
    
    if pr:
        print('VaR_95:')
        print(t,' Kupiec:', res['Ku95'])
        print(t,'Christ.:', res['Ch95'])
        print(t,'Scoring:', )
        print('VaR_99:')
        print(t,' Kupiec:', res['Ku99'])
        print(t,'Christ.:', res['Ch99'])
        print(t,'Scoring:', np.mean(scoring_quantile(r_test, v99, .099)))
        print('EVaR_95:')
        print(t,'Scoring:', np.mean(scoring_expectile(r_test, ev95,.95)))
        print('EVaR_99:')
        print(t,'Scoring:', np.mean(scoring_expectile(r_test, ev99,.99)))
    return(res)


In [83]:
dst = (
"genhyperbolic" ,
'mielke' ,
"johnsonsu" ,
"nct" ,
#'burr' ,
"genlogistic" ,
"norminvgauss" ,
'burr12' ,
#"crystalball" ,
"t" ,
'fisk' ,
"logistic" ,
"tukeylambda" ,
'powernorm' ,
"powerlognorm" ,
"exponweib" ,
"gengamma" ,
"loggamma" ,
"hypsecant" ,
"dweibull" )

results_df=pd.DataFrame()
for dist_type in dst:
    print('')
    print(dist_type)
    filename = './data_distfit/fitted_{}.csv'.format(dist_type)
    df1 = pd.read_csv(filename, index_col=0)
    a = summary_backtesting(df1)
    b =pd.DataFrame.from_dict(a,orient='index',columns=[dist_type])
    results_df = pd.concat([results_df, b.transpose()], ignore_index=False)




genhyperbolic
VaR_95:
         Kupiec: 0.007447710138538399
        Christ.: 0.6492634807020252
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.052556951236776686
EVaR_95:
        Scoring: 0.00015073018772343078
EVaR_99:
        Scoring: 5.2472696450239176e-05

mielke
VaR_95:
         Kupiec: 0.0001276037136812569
        Christ.: 0.839492577936688
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.6739419234923614
EVaR_95:
        Scoring: 0.43364530712981836
EVaR_99:
        Scoring: 1.050706445279327

johnsonsu
VaR_95:
         Kupiec: 0.007447710138538399
        Christ.: 0.6492634807020252
        Scoring:
VaR_99:
         Kupiec: 0.18099777733592193
        Christ.: 0.9946240358584997
        Scoring: 0.05274791301614226
EVaR_95:
        Scoring: 0.00015213245653774413
EVaR_99:
        Scoring: 5.460605562374835e-05

nct
VaR_95:
         Kupiec: 

In [84]:
b = pd.DataFrame.from_dict(a,orient='index',columns=['t'])

In [85]:
results_df

,Ku95,Ku99,Ch95,Ch99,Sc95,Sc99,ESc95,ESc99
genhyperbolic,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002148,0.000594,0.000151,0.000052
mielke,1.276037e-04,1.809978e-01,0.839493,0.994624,0.012401,0.007495,0.433645,1.050706
johnsonsu,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002151,0.000594,0.000152,0.000055
nct,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002151,0.000594,0.000152,0.000055
genlogistic,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002162,0.000609,0.000155,0.000055
norminvgauss,7.447710e-03,1.809978e-01,0.649263,0.994624,0.002154,0.000594,0.000152,0.000053
burr12,1.276037e-04,1.809978e-01,0.839493,0.994624,0.002199,0.000611,0.000163,0.000053
crystalball,6.270683e-04,5.440159e-01,0.774289,0.906913,0.002178,0.000586,0.000145,0.000045
t,1.276037e-04,1.438720e-02,0.839493,NaN,0.002266,0.000668,0.000189,0.000075
fisk,3.219016e-08,1.438720e-02,NaN,NaN,0.047280,0.042945,7.076557,25.149875


In [86]:
wybrane = np.array(("genhyperbolic", "johnsonsu", 't'))

In [87]:
results_df.transpose()[wybrane]

,genhyperbolic,johnsonsu,t
Ku95,0.007448,0.007448,0.000128
Ku99,0.180998,0.180998,0.014387
Ch95,0.649263,0.649263,0.839493
Ch99,0.994624,0.994624,NaN
Sc95,0.002148,0.002151,0.002266
Sc99,0.000594,0.000594,0.000668
ESc95,0.000151,0.000152,0.000189
ESc99,0.000052,0.000055,0.000075


In [88]:
results_df.to_csv('./data/backtesting.csv')

In [89]:
results_df.index

Index(['genhyperbolic', 'mielke', 'johnsonsu', 'nct', 'genlogistic',
       'norminvgauss', 'burr12', 'crystalball', 't', 'fisk', 'logistic',
       'tukeylambda', 'powernorm', 'powerlognorm', 'exponweib', 'gengamma',
       'loggamma', 'hypsecant', 'dweibull'],
      dtype='object')